In [2]:
import pandas as pd
from pyspi.calculator import Calculator
import numpy as np
from copy import deepcopy
import glob
import os
import random
import neuromaps
from neuromaps.parcellate import Parcellater
from nibabel import freesurfer as fs
import nibabel as nib
from neuromaps.datasets import fetch_annotation
from pathlib import Path
from scipy.spatial.distance import cdist
from scipy.stats import spearmanr
import statsmodels.stats.multitest

# Set seed to 127
random.seed(127)

# Add connectome workbench to path
os.environ['PATH'] = os.environ['PATH'] + ':/Applications/workbench/bin_macosx64'

%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [3]:
%%R 

suppressPackageStartupMessages({
    library(broom)
    library(circlize)
    library(ComplexHeatmap)
    library(cowplot)
    library(dendextend)
    library(FactoMineR)
    library(GGally)
    library(ggseg)
    library(glue)
    library(grid)
    library(patchwork)
    library(see)
    library(tidyverse)
})

# Set cowplot theme
theme_set(theme_cowplot())

## Load in data

In [42]:
data_path = "/Users/abry4213/data/HCP100/"

# Load in pyspi SPI info 
pyspi_info = pd.read_csv("../feature_extraction/pyspi_SPI_info.csv")

# Load in module colours
module_colours = (pd.read_csv("SPI_module_colours.csv")
                  .query("Module_Type == 'HClust'")
                  .assign(Module = lambda x: x.Module.str.replace("M", "").astype('int'))
                  )

# Load in all the pyspi homotopic res
all_homotopic_res = (pd.concat([pd.read_csv(f) for f in glob.glob(data_path + "time_series_features/pyspi/*.csv")])
                     .merge(pyspi_info, on="SPI", how="left")
                              .groupby(['base_region_to', 'SPI', 'Subject'])['value']
                              .mean()
                              .reset_index()
                              .dropna(subset=['value'])
                              )


In [9]:
subject_wise_SPI_corrs = (all_homotopic_res
                  .pivot_table(index=['base_region_to', 'Subject'], columns='SPI', values='value')
                    .reset_index()
                    .drop(columns=['base_region_to'])
                    .set_index('Subject')
                    .groupby('Subject')
                    .apply(lambda x: x.corr(method='spearman'))
                  )

In [16]:
mean_SPI_corrs_across_subjects = (subject_wise_SPI_corrs
 .reset_index()
 .melt(id_vars=['Subject', 'SPI'], var_name='SPI2', value_name='Spearman_Rho')
 .dropna(subset=['Spearman_Rho'])
 .rename(columns={'SPI': 'SPI1'})
 .groupby(['SPI1', 'SPI2'])['Spearman_Rho']
    .mean()
    .reset_index()
 )

mean_SPI_corrs_across_subjects_wide = mean_SPI_corrs_across_subjects.pivot(index='SPI1', columns='SPI2', values='Spearman_Rho')

In [36]:
%%R -i mean_SPI_corrs_across_subjects_wide -o SPI_performance_cluster_df

library(ComplexHeatmap)

svg("../plots/SPI_similarity/All_SPI_spearman_corr_heatmap.svg", width=5, height=5, bg=NA)
ht1 <- ComplexHeatmap::Heatmap(mean_SPI_corrs_across_subjects_wide,
                                clustering_method_rows = "ward.D2",
                                clustering_method_columns = "ward.D2",
                                row_names_side = "right",
                                row_dend_side = "left", 
                                row_dend_width = unit(1, "cm"),
                                row_dend_gp = gpar(lwd=unit(0.5, "cm")),
                                row_split = 5,
                                column_split = 5,
                                row_title = NULL,
                                column_title = NULL,
                                show_row_names = FALSE,
                                show_column_names = FALSE,
                                name = "Spearman corr",
                                show_column_dend = FALSE,
                                heatmap_legend_param = list(legend_direction = "horizontal",
                                                            legend_width = unit(5, "cm"))) 

ht1 <- draw(ht1, heatmap_legend_side = "bottom",
    padding = unit(c(2, 2, 2, 2), "mm"),
    background = "transparent")

dev.off()

# Get row order
row_order <- row_order(ht1)
SPI_index_df <- data.frame(SPI_Index = 1:length(rownames(mean_SPI_corrs_across_subjects_wide)), 
                           SPI = rownames(mean_SPI_corrs_across_subjects_wide))

# Map the index of the row order as a cluster, to each node
# Convert to data frame
SPI_performance_cluster_df <- do.call(rbind, lapply(seq_along(row_order), function(i) {
  data.frame(SPI_Index = row_order[[i]], Cluster = i)
})) %>% left_join(SPI_index_df)


Joining with `by = join_by(SPI_Index)`


In addition: Warning message:
The input is a data frame-like object, convert it to a matrix. 


In [33]:
SPI_performance_cluster_df_info = SPI_performance_cluster_df.merge(pyspi_info, on='SPI', how='left')
SPI_performance_cluster_df_info

,SPI_Index,Cluster,SPI,Directionality,Literature_category,Module,Sonnet,Figure_name
0,148,1,plv_multitaper_mean_fs-1_fmin-0_fmax-0-5,Undirected,spectral,M10,NaN,NaN
1,26,1,cohmag_multitaper_mean_fs-1_fmin-0_fmax-0-5,Undirected,spectral,M12,Yes,coherence_mag
2,154,1,ppc_multitaper_mean_fs-1_fmin-0_fmax-0-5,Undirected,spectral,M12,NaN,NaN
3,3,1,bary-sq_dtw_mean,Undirected,distance,NaN,NaN,NaN
4,202,1,si_gaussian_k-1,Undirected,infotheory,M05,NaN,NaN
...,...,...,...,...,...,...,...,...
229,100,5,icoh_multitaper_mean_fs-1_fmin-0-25_fmax-0-5,Undirected,spectral,M12,NaN,NaN
230,139,5,phase_multitaper_mean_fs-1_fmin-0_fmax-0-25,Undirected,spectral,M01,NaN,NaN
231,140,5,phase_multitaper_mean_fs-1_fmin-0_fmax-0-5,Undirected,spectral,M01,NaN,NaN
232,98,5,icoh_multitaper_max_fs-1_fmin-0_fmax-0-25,Undirected,spectral,M12,NaN,NaN


In [52]:
%%R -i SPI_performance_cluster_df_info,module_colours

# Convert to named vector
colormap <- setNames(module_colours$Colour, module_colours$Module)

SPI_performance_cluster_df_info %>% 
    group_by(Cluster, Module) %>%
    count() %>% 
    mutate(Module_Num = as.numeric(gsub("M", "", Module))) %>%
    mutate(Module_Num = factor(Module_Num, levels=1:14)) %>%
    ggplot(aes(x = Module_Num, y = n, fill = Module_Num)) +
    scale_fill_manual(values=colormap, na.value='gray90') +
    geom_bar(stat = "identity") +
    xlab("pyspi module") +
    facet_grid(Cluster ~ ., scales='free') +
    theme(strip.background = element_blank(),
          strip.text.y = element_text(angle=0),
          legend.position = "none") 
ggsave("../plots/SPI_similarity/SPI_performance_cluster_module_barplot.svg", width=6, height=4.5, units="in", dpi=300)

/Users/abry4213/miniforge3/envs/annie_env/lib/python3.9/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "Sonnet". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '
/Users/abry4213/miniforge3/envs/annie_env/lib/python3.9/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "Figure_name". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '
